In [ ]:
import sys
import math
import time
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
from seaborn import histplot
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time
from ctapipe.io import EventSource, EventSeeker
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
from calibpipe.throughput import MuonCalibrator
import h5py
from astropy.io import fits






In [ ]:
# Read all energies of the simulation
energies_all = []
counter = 0
for k in range(1,11):
    filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        counter += 1
        energies_all.append(j.simulation.shower.energy)

In [ ]:
histplot(energies_all, log_scale = True, bins = 45)

In [ ]:
# Create event container with all events
event_container = np.array([])
for k in range(1,2):
    filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,j in enumerate(event_iterator):
        event_container = np.append(event_container, j)

In [ ]:
event_container[0].simulation.shower.energy.to_value()

In [ ]:
###
### To calibrate events and process muon rings
###
int_cut = 0.3
min_pixel = 80
muons = []

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})


for k in range(1,11):
    filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):           
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        calib(event)
        image_processor(event)
        muon_processor(event)
        if not np.isnan(event.muon.tel[1].efficiency.optical_efficiency):
            muons.append(event)

# optical efficiency of muons           
opt_eff = []
for muon_event in muons:
    opt_eff.append(muon_event.muon.tel[1].efficiency.optical_efficiency)        

In [ ]:
# Not cleaned dataset
print(np.mean(opt_eff))
print(np.std(opt_eff))
print(len(opt_eff))

In [ ]:
####  NEW CLEANING NEW CLEANING NEW CLEANING
####
####
####

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels < {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

int_cut = 0.2
min_pixel = 200
muons_cleaned = []
tels = 1

for k in range(1,11):
    filename = f'/Users/vdk/output_lst1_12Septv1/corsika_theta_0.0_az_180.0_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator): 
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        pixel_mask = []
        for k,l in enumerate(event.r0.tel[tels].waveform[1]):
            if np.argmax(l) in range(16,40): 
                pixel_mask.append(True)
            else:
                pixel_mask.append(False)
        working_wave = event.r1.tel[tels].waveform.copy()
        for k,l in enumerate(working_wave):
            if pixel_mask[k]:
                #working_wave[k] = np.zeros(40)     
                working_wave[k] = np.mean(event.r1.tel[tels].waveform, axis = 0)
        event.r1.tel[tels].waveform = working_wave
    
        calib(event)
        image_processor(event)
        muon_processor(event)
        if not np.isnan(event.muon.tel[tels].efficiency.width):
            muons_cleaned.append(event)
      


In [ ]:
# Processing of cleaned images, to choose proper muon rings
true_muons_cleaned = []

for event in muons_cleaned:
    if event.dl1.tel[tels].parameters.morphology.n_pixels > 30 and abs(event.muon.tel[tels].parameters.intensity_ratio - 1) < 0.3 and event.muon.tel[tels].parameters.completeness > 0.4 and event.muon.tel[tels].parameters.mean_squared_error.to_value() < 0.01:
        print(event.index.event_id)
        plt.figure(figsize = (9,6))
        camgeom = source.subarray.tel[tels].camera.geometry
        title=f"CleanedRing{event.index.event_id}_optEff{round(event.muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(event.muon.tel[tels].efficiency.width.to_value(),3)}"
        disp = CameraDisplay(camgeom,title=title)
        disp.image = event.simulation.tel[tels].true_image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #plt.show()
        true_muons_cleaned.append(event)
        
plt.close()

cleaned_opt_eff = []
for event in true_muons_cleaned:
    cleaned_opt_eff.append(event.muon.tel[1].efficiency.optical_efficiency)

In [ ]:
# Cleaned dataset
print(np.mean(cleaned_opt_eff))
print(np.std(cleaned_opt_eff))
print(len(cleaned_opt_eff))

In [ ]:
cleaned_completeness = []
for i in true_muons_cleaned:
    cleaned_completeness.append(i.muon.tel[1].parameters.completeness)

completeness = []
for i in muons:
    completeness.append(i.muon.tel[1].parameters.completeness)

plt.hist(completeness,bins = 7, label = 'completeness of muon rings', histtype = 'step')
plt.hist(cleaned_completeness,bins = 7, alpha = 1, label = 'completeness of cleaned muon rings', histtype = 'step')
plt.xlabel("counts")
plt.ylabel("completeness")
plt.legend()

Lets work with hdf5 files

In [ ]:
filename = '/Users/vdk/Downloads/DL1_datacheck_20220628.h5'
fi = h5py.File(filename, 'r')
keys = list(fi.keys())

def printname(name):
    print(name)

# Show all included tables
fi.visit(printname)

In [ ]:
for item in fi['runsummary/_i_table/elapsed_time']:
    print(item)

Now Fits format

In [ ]:
# Test table we received as result of script lstchain_dl1_muon_analysis.py
filename_fits = '~/folder_for_test/test_table.fits'
hdul = fits.open(filename_fits)
hdul.info()
#hdul[1].data

In [ ]:
# element number 10 is muon efficiency, so in such way we can check it
for i in range(len(hdul[1].data)):
    if not np.isnan(hdul[1].data[i][10]):
        print(hdul[1].data[i])

In [ ]:
counter = 100
np.savetxt(f'/Users/vdk/values.txt', [counter])

In [ ]:
int_cut = 0.3

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})
muons = []

for run in range(1001, 1100):
    crab_sim_file = f'/Users/vdk/crab_sim/corsika_theta_16.087_az_108.090_run{run}.simtel.gz'
    source = EventSource(crab_sim_file)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):           
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        calib(event)
        image_processor(event)
        muon_processor(event)
        if not np.isnan(event.muon.tel[1].efficiency.optical_efficiency):
            muons.append(event)

In [ ]:
opt_eff = []
for event in muons:
    opt_eff.append(event.muon.tel[1].efficiency.optical_efficiency)

np.mean(opt_eff)

In [ ]:
tels = 1
for event in muons:
    print(event.index.event_id)
    plt.figure(figsize = (9,6))
    camgeom = source.subarray.tel[tels].camera.geometry
    title=f"CleanedRing{event.index.event_id}_optEff{round(event.muon.tel[tels].efficiency.optical_efficiency,3)}_width{round(event.muon.tel[tels].efficiency.width.to_value(),3)}"
    disp = CameraDisplay(camgeom,title=title)
    disp.image = event.simulation.tel[tels].true_image
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    #plt.show()

In [ ]:
len(opt_eff)

CRAB LST 1 hour (3 runs approx. 20 min each) OBSERVATIONS 

In [ ]:
crab_obs_file_1 = '~/crab_obs/muons_LST-1.Run10880.fits'
crab_obs_file_2 = '~/crab_obs/muons_LST-1.Run10881.fits'
crab_obs_file_3 = '~/crab_obs/muons_LST-1.Run10882.fits'

hdul_crab_1 = fits.open(crab_obs_file_1)
hdul_crab_2 = fits.open(crab_obs_file_2)
hdul_crab_3 = fits.open(crab_obs_file_3)
hdul_crab_1.info()

In [ ]:
len(hdul_crab_1[1].data)

In [ ]:
len(hdul_crab_2[1].data)

In [ ]:
len(hdul_crab_3[1].data)

In [ ]:
len(hdul_crab_1[1].data) + len(hdul_crab_2[1].data) + len(hdul_crab_3[1].data)

In [ ]:
flag_count = 0
for flag in hdul_crab_1[1].data:
    if flag:
        flag_count += 1

In [ ]:
flag_count

In [ ]:
filename1 = '/Users/vdk/energies_500.txt'
fi1 = open(filename1,'r')
lines = fi1.readlines() 
splitted = lines[0].split(',')
energies_proton_1 = np.array(list(filter(None, splitted))).astype('float')
fi1.close()

filename1 = '/Users/vdk/energies_501-1100.txt'
fi1 = open(filename1,'r')
lines = fi1.readlines() 
splitted = lines[0].split(',')
energies_proton_2 = np.array(list(filter(None, splitted))).astype('float')
fi1.close()

filename1 = '/Users/vdk/energies_625-1100.txt'
fi1 = open(filename1,'r')
lines = fi1.readlines() 
splitted = lines[0].split(',')
energies_proton_3 = np.array(list(filter(None, splitted))).astype('float')
fi1.close()

energies_proton = np.append(energies_proton_1, energies_proton_2)
energies_proton = np.append(energies_proton, energies_proton_3)

filename2 = '/Users/vdk/energies_for_filerange1-1100.txt'
fi2 = open(filename2, 'r')
lines = fi2.readlines()
energies_muon = np.array(lines).astype('float')



w_protons = sum([energy**(-0.7) for energy in energies_proton])
new_proton_sample = len(energies_proton)/ w_protons  * len(energies_proton)
new_proton_sample = w_protons

print("Initial proton sample size = ", len(energies_proton))
print("Weighted proton sample size = ", new_proton_sample)

w_muons = sum([energy**(-0.7) for energy in energies_muon])
new_muon_sample = len(energies_muon)/ w_muons * len(energies_muon)
new_muon_sample = w_muons

print("Initial muon sample size = ", len(energies_muon))
print("Weighted muon sample size = ", new_muon_sample)

print("Initial relation between muon sample size and proton sample = ", len(energies_muon)/len(energies_proton))
print("Weighed relation between muon sample size and proton sample = ", new_muon_sample / new_proton_sample)

print("reweigth muons = ", w_muons / len(energies_muon))
print("reweigth protons = ", w_protons / len(energies_proton))

In [ ]:
plt.hist(energies_proton, log=True, bins = 30)

In [ ]:
plt.hist([energy**(-0.7) for energy in energies_proton], log=True, bins = 30)

In [ ]:
plt.hist([energy**(-0.7) for energy in energies_muon], log=True, bins = 10)

In [ ]:
plt.hist(energies_muon, log=True, bins = 10)

In [ ]:
filename1 = '/Users/vdk/energies_500.txt'
fi1 = open(filename1,'r')
lines = fi1.readlines() 
splitted = lines[0].split(',')
energies_proton = np.array(list(filter(None, splitted))).astype('float')
fi1.close()

filename2 = '/Users/vdk/energies_for_filerange1-499.txt
fi2 = open(filename2, 'r')
lines = fi2.readlines()
energies_muon = np.array(lines).astype('float')



w_protons = sum([energy**(-0.7) for energy in energies_proton])
new_proton_sample = len(energies_proton)/ w_protons  * len(energies_proton)
new_proton_sample = w_protons

print("Initial proton sample size = ", len(energies_proton))
print("Weighted proton sample size = ", new_proton_sample)

w_muons = sum([energy**(-0.7) for energy in energies_muon])
new_muon_sample = len(energies_muon)/ w_muons * len(energies_muon)
new_muon_sample = w_muons

print("Initial muon sample size = ", len(energies_muon))
print("Weighted muon sample size = ", new_muon_sample)

print("Initial relation between muon sample size and proton sample = ", len(energies_muon)/len(energies_proton))
print("Weighed relation between muon sample size and proton sample = ", new_muon_sample / new_proton_sample)

print("reweigth muons = ", w_muons / len(energies_muon))
print("reweigth protons = ", w_protons / len(energies_proton))